In [ ]:
#
# Martin Boldt, Blekinge Institute of Technology, Oct 2019.
#

import pandas as pd
import numpy as np

### Läs in datan från filen "Grilla.csv" till en Pandas Dataframe och skriv ut datan på skärmen. Tolka varje kolumn som kategorisk data (engelska: 'category')

In [ ]:
df = pd.read_csv("Grilla.csv",  dtype='category', sep=",")
print( df )

### Skriv ut kodningen av kategorierna för respektive kolumn


In [ ]:
for catindex, catname in enumerate( df.columns ):
    print( "{0:10s} = {1}".format( catname, df[catname].cat.categories ) )

### Spara integer-representation av kategorierna (för respektive kolumn) i en ny dataframe, och skriv ut deras koder på skärmen


In [ ]:
df_coded = pd.DataFrame()

df_coded['Dag'] = df['Dag'].astype('category').cat.codes
df_coded['Väder'] = df['Väder'].astype('category').cat.codes
df_coded['Temperatur'] = df['Temperatur'].astype('category').cat.codes
df_coded['Fuktighet'] = df['Fuktighet'].astype('category').cat.codes
df_coded['Vind'] = df['Vind'].astype('category').cat.codes
df_coded['Grilla'] = df['Grilla'].astype('category').cat.codes

print( df_coded )

### Spara datan i samtliga kolumner _utom_ första och sista i dataframen "X". Spara därefter sista kolumen (alltså klassvariablen) till dataframen "y"


In [ ]:
X = df_coded.iloc[:,1:5] 
y = df_coded.iloc[:,5]

### Skapa ett beslutsträd med följande argument satta: criterion="entropy" samt random_state"=0. Bygg därefter beslutsträdet baserat på datan i "X" och "y" med hjälp av beslutsträdets funktion fit()


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier(criterion='entropy', random_state=0)
dtree.fit(X,y)

### Skriv ut trädet till en PNG-fil.

In [ ]:
# Generera svartvit trädstruktur till PNG-fil
from graphviz import Source
from sklearn.tree import export_graphviz

graph = Source( export_graphviz(dtree, out_file=None, feature_names=X.columns))
graph.format = 'png'
graph.render('grilla_tree',view=True)

In [ ]:
# Generarea snyggare färglagd trädstruktur (vita noder, negativa löv som röda och positiva lov som gröna) till PNG-fil.
import pydotplus

dot_data = export_graphviz(dtree, feature_names=X.columns, out_file=None, filled=True, rounded=True)
graph = pydotplus.graph_from_dot_data(dot_data)
nodes = graph.get_node_list()
colors =  ('springgreen', 'tomato', 'white')

for node in nodes:
    if node.get_name() not in ('node', 'edge'):
        values = dtree.tree_.value[int(node.get_name())][0]
        #color only nodes where only one class is present
        if max(values) == sum(values):    
            node.set_fillcolor(colors[np.argmax(values)])
        #mixed nodes get the default color
        else:
            node.set_fillcolor(colors[-1])
graph.write_png('grilla_tree_color.png')

### Skapa en ny instans som har följande värden: "Väder"=0, "Temperatur"=0, "Fuktighet"=0, "Vind"=0

In [ ]:
test = pd.DataFrame( [[0,0,0,0]], columns = [ "Väder", "Temperatur", "Fuktighet", "Vind" ] )
print( test.to_string(index=False) )

### Klassificera den nya instansen m.h.a. beslutsträdet och skriv ut vilken klass den får. Förklara varför instansen klassifieras som den gör. Vilken väg genom trädet används?

In [ ]:
test_pred = dtree.predict(test)
print( test_pred )

### Skapa en ny instans som har följande värden: "Väder"=2, "Temperatur"=2, "Fuktighet"=1, "Vind"=1

In [ ]:
test = pd.DataFrame( [[2,2,1,1]], columns = [ "Väder", "Temperatur", "Fuktighet", "Vind" ] )
print( test.to_string(index=False) )

### Klassificera den nya instansen m.h.a. beslutsträdet och skriv ut vilken klass den får. Förklara varför instansen klassifieras som den gör. Vilken väg genom trädet används?

In [ ]:
test_pred = dtree.predict(test)
print( test_pred )